In [58]:
'''
In this file, we calculate the von neumann entropy of a series of diagrams. 

We compare them to each other in the hope of finding that more ambiguous (less disambiguated sentences) have higher entropy. 

'''



'\nIn this file, we calculate the von neumann entropy of a series of diagrams. \n\nWe compare them to each other in the hope of finding that more ambiguous (less disambiguated sentences) have higher entropy. \n\n'

In [59]:
'''
start by importing all sorts of stuff
'''


import numpy as np
from sympy import Symbol, symbols

from lambeq.ansatz import BaseAnsatz
from lambeq.backend.grammar import Box, Diagram, Functor, Ty
from lambeq.backend.quantum import (
    Bra,
    CRz,
    Diagram as Circuit,
    Discard,
    H,
    CX,
    Id,
    Ket,
    quantum,
    qubit,
    Rotation,
    Rx, Ry, Rz
)

from lambeq import AtomicType


In [60]:
'''
Note that for this program, we do not have to import test data. 
We test the entropy for sentences crafted in the program especially for the program. 

We write three functions for three types of sentences: 
1. The normal way of parsing 
2. Alice runs and Bob does too 
3. Alice cooks food and eats it. 
'''

from lambeq import BobcatParser
parser = BobcatParser(verbose='text')
from lambeq import RemoveCupsRewriter
remove_cups = RemoveCupsRewriter()
from lambeq import AtomicType, IQPAnsatz
ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1},
                   n_layers=1, n_single_qubit_params=3)

from lambeq.backend.grammar import (Box, Cap, Cup, Diagram, Diagrammable,
                                    Functor, grammar, Id, Spider, Swap,
                                    Ty, Word)
from lambeq import AtomicType
N = AtomicType.NOUN
S = AtomicType.SENTENCE
n, s = Ty('n'), Ty('s')

# create three functions for three types of diagrams

def GijsDiag(sentence:str): 
    '''
    This function, given a sentence of the form: 
    Subj Verb Obj and Subj* does too
    , returns a diagram that copies the information in such a way that the returned diagram reads: 
    Subj Verb Obj and Subj* Verb Obj
    The object will be removed later to work with density matrices. 
    '''
    wordList = sentence.split()
    grammar = (Spider(n, 2, 1) @ Id(n.r) @ Id(s) @ Cup(n.l, n) >> 
            Cup(n, n.r) @ Id(s) )
    newWords = [
        Word(wordList[0], n),
        # Word('test', n),
        Word(wordList[4], n),
        Word(wordList[1], n.r @ s @ n.l), 
        Word(wordList[2], n)
    ]

    diagram = Id().tensor(*newWords) >> grammar
    diagram = remove_cups(diagram)
    # diagram.draw()

    circuit = ansatz(diagram= diagram)

    return circuit#, diagram 

def NaiveDiag(sentence:str):
    '''
    Simply returns a naive circuit given a diagram
    '''
    diagram = parser.sentence2diagram(sentence= sentence, suppress_exceptions=True)

    diagram = remove_cups(diagram)

    # diagram.draw()

    circuit = ansatz(diagram= diagram)

    return circuit#, diagram

def AmbiguousVerb(sentence:str):
    '''
    This function, given a sentence like: 
    Subj Verb Obj and Verb* it. (Man prepares dinner and eats it.) 
    returns a diagram that copies the information, so that: 
    Subj Verb Obj and Subj Verb* Obj
    The object will be removed later 
    '''
    wordList = sentence.split()
    grammar = (Spider(n, 1, 2) @ Id(n.r) @ Id(s) @ Id(n.l) @ Spider(n, 1, 2)  >> 
                Id(n) @ Cup(n, n.r) @ Id(s) @ Cup(n.l, n) @ Id(n) >> 
                Id(n) @ Word(wordList[4], n.r @ s @ n.l) @ Id(s) @ Id(n) >> 
                Cup(n, n.r) @ Id(s) @ Swap(n.l, s) @ Id(n) >> 
                Spider(s, 2, 1) @ Cup(n.l, n))
    newWords = [
        Word(wordList[0], n),
        Word(wordList[1], n.r @ s @ n.l),
        Word(wordList[2], n)
    ]


    diagram = Id().tensor(*newWords) >> grammar
    diagram = remove_cups(diagram)
    # diagram.draw()
    circuit = ansatz(diagram= diagram)

    return circuit#, diagram


    




In [61]:
def AmbiguousVerb2(sentence:str): 
    '''
    
    '''
    wordList = sentence.split()
    newWords = [
        # Word(wordList[0], n),
        Word(wordList[1], n.r @ s @ n.l),
        Word(wordList[4], n.r @ s @ n.l),
        # Word(wordList[2], n)
    ]

    grammar = (
        Id(n.r) @ Id(s) @ Swap(n.l, n.r) @ Id(s) @ Id(n.l) >> 
        Id(n.r) @ Swap(s, n.r) @ Swap(n.l, s) @ Id(n.l) >> 
        Spider(n.r, 2, 1) @ Spider(s, 2, 1) @ Spider(n.l, 2, 1) >> 
        Word(wordList[0], n.r).dagger() @ Id(s) @ Word(wordList[2], n.l).dagger()
    )


    # grammar = (
    #     Spider(n, 1, 2) @ Id(n.r) @ Id(s) @ Id(n.l) @ Id(n.r) @ Id(s) @ Id(n.l) @ Spider(n, 1, 2) >>
    #     Id(n) @ Cup(n, n.r) @ Id(s) @ Id(n.l) @ Id(n.r) @ Id(s) @ Cup(n.l, n) @ Id(n) 
    # )

    diagram = Id().tensor(*newWords) >> grammar
    # diagram = remove_cups(diagram)
    diagram.draw()
    circuit = ansatz(diagram= diagram)

    return circuit#, diagram

In [62]:

GIJS = True  

# naiveSentences = ['person prepares meal', 'person cooks meal', 'person bakes meal', 'person reads meal', 'person runs meal'
#                 'person prepares dinner', 'person cooks dinner', 'person bakes dinner', 'person reads dinner', 'person runs dinner',
#                 'person prepares sauce', 'person cooks sauce', 'person bakes sauce', 'person reads sauce', 'person runs sauce',
#                 'person prepares program', 'person cooks program', 'person bakes program', 'person reads program', 'person runs program',
#                 'person prepares software', 'person cooks software', 'person bakes software', 'person reads software', 'person runs software',
#                 'person prepares application', 'person cooks application', 'person bakes application', 'person reads application', 'person runs application',
#                 ]





if not GIJS: 

    senseFulSentences = ['person prepares meal', 'person cooks meal', 'person bakes meal',
                    'person prepares dinner', 'person cooks dinner', 'person bakes dinner',
                    'person prepares sauce', 'person cooks sauce', 'person bakes sauce',
                    'person prepares program', 'person reads program', 'person runs program',
                    'person prepares software', 'person reads software', 'person runs software',
                    'person prepares application', 'person reads application', 'person runs application',
                    'man prepares meal', 'man cooks meal', 'man bakes meal',
                    'man prepares dinner', 'man cooks dinner', 'man bakes dinner',
                    'man prepares sauce', 'man cooks sauce', 'man bakes sauce',
                    'man prepares program', 'man reads program', 'man runs program',
                    'man prepares software', 'man reads software', 'man runs software',
                    'man prepares application', 'man reads application', 'man runs application',
                    'woman prepares meal', 'woman cooks meal', 'woman bakes meal',
                    'woman prepares dinner', 'woman cooks dinner', 'woman bakes dinner',
                    'woman prepares sauce', 'woman cooks sauce', 'woman bakes sauce',
                    'woman prepares program', 'woman reads program', 'woman runs program',
                    'woman prepares software', 'woman reads software', 'woman runs software',
                    'woman prepares application', 'woman reads application', 'woman runs application',
                    ]
    
    # mySentences = ['person prepares meal and cooks it', 'person prepares meal and bakes it', 'person prepares meal and reads it', 'person prepares meal and runs it',
    #                 'man prepares meal and cooks it', 'man prepares meal and bakes it', 'man prepares meal and reads it', 'man prepares meal and runs it',
    #                 'woman prepares meal and cooks it', 'woman prepares meal and bakes it', 'woman prepares meal and reads it', 'woman prepares meal and runs it',
    #                 'person prepares meal and cooks it', 'person prepares meal and bakes it', 'person prepares meal and reads it', 'person prepares meal and runs it',
    #                 'man prepares meal and cooks it', 'man prepares meal and bakes it', 'man prepares meal and reads it', 'man prepares meal and runs it',
    #                 'woman prepares meal and cooks it', 'woman prepares meal and bakes it', 'woman prepares meal and reads it', 'woman prepares meal and runs it']


    mySentences = ['person prepares meal and cooks it', 'person prepares meal and bakes it',
                    'person prepares dinner and cooks it', 'person prepares dinner and bakes it',
                    'person prepares sauce and cooks it', 'person prepares sauce and bakes it',
                    'person prepares program and runs it', 'person prepares program and reads it',
                    'person prepares software and runs it', 'person prepares software and reads it',
                    'person prepares application and runs it', 'person prepares application and reads it',
                    'woman prepares meal and cooks it', 'woman prepares meal and bakes it',
                    'woman prepares dinner and cooks it', 'woman prepares dinner and bakes it',
                    'woman prepares sauce and cooks it', 'woman prepares sauce and bakes it',
                    'woman prepares program and runs it', 'woman prepares program and reads it',
                    'woman prepares software and runs it', 'woman prepares software and reads it',
                    'woman prepares application and runs it', 'woman prepares application and reads it',
                    'man prepares meal and cooks it', 'man prepares meal and bakes it',
                    'man prepares dinner and cooks it', 'man prepares dinner and bakes it',
                    'man prepares sauce and cooks it', 'man prepares sauce and bakes it',
                    'man prepares program and runs it', 'man prepares program and reads it',
                    'man prepares software and runs it', 'man prepares software and reads it',
                    'man prepares application and runs it', 'man prepares application and reads it',
                    ]

# gijs doesn't know words man, woman 
if GIJS:
    senseFulSentences = ['person prepares meal', 'person cooks meal', 'person bakes meal',
                    'person prepares dinner', 'person cooks dinner', 'person bakes dinner',
                    'person prepares sauce', 'person cooks sauce', 'person bakes sauce',
                    'person prepares program', 'person reads program', 'person runs program',
                    'person prepares software', 'person reads software', 'person runs software',
                    'person prepares application', 'person reads application', 'person runs application',
                    'chef prepares meal', 'chef cooks meal', 'chef bakes meal',
                    'chef prepares dinner', 'chef cooks dinner', 'chef bakes dinner',
                    'chef prepares sauce', 'chef cooks sauce', 'chef bakes sauce',
                    'programmer prepares program', 'programmer reads program', 'programmer runs program',
                    'programmer prepares software', 'programmer reads software', 'programmer runs software',
                    'programmer prepares application', 'programmer reads application', 'programmer runs application'
                    ]



    mySentences = ['person prepares meal and cooks it', 'person prepares meal and bakes it', 'person prepares meal and reads it', 'person prepares meal and runs it', 
                   'person prepares meal and cooks it', 'person prepares meal and bakes it', 'person prepares meal and reads it', 'person prepares meal and runs it']

    mySentences = ['person prepares meal and cooks it', 'person prepares meal and bakes it',
                    'person prepares dinner and cooks it', 'person prepares dinner and bakes it',
                    'person prepares sauce and cooks it', 'person prepares sauce and bakes it',
                    'person prepares program and runs it', 'person prepares program and reads it',
                    'person prepares software and runs it', 'person prepares software and reads it',
                    'person prepares application and runs it', 'person prepares application and reads it']



# we disambiguate the first sentence using the second one 
gijsSentences = ['person prepares meal and chef does too', 'person prepares dinner and chef does too', 'person prepares sauce and chef does too',
                 'person prepares application and programmer does too', 'person prepares software and programmer does too', 'person prepares program and programmer does too']
                #  ]#, 'person prepares meal and person does too']



naiveCircuits = []
gijsCircuits  = []
myCircuits    = []


for nS in senseFulSentences: 
    naiveCircuits.append(NaiveDiag(nS))

for gS in gijsSentences: 
    gijsCircuits.append(GijsDiag(gS))

for mS in mySentences: 
    myCircuits.append(AmbiguousVerb(mS))


  

In [63]:
# for circuit in naiveCircuits: 
#     circuit.draw()
# gijsDiags[0].draw()
# gijsCircuits[0].draw()
# myCircuits[0].draw()
# naiveCircuits[0].draw()

In [64]:
'''
import the functions needed to set up the diagrams using the trained weights 
'''

from entropyHelpers import *


In [65]:
'''
import trained weights 
'''
# one of these two is wordsToForget
wordsToForgetSubj = ['man', 'woman', 'person', 'chef', 'programmer']
wordsToForgetObj  = ['dinner', 'meal', 'sauce', 'program', 'application', 'software']


# don't use these words together with chef or something. 
# do not use words in the same list taht appear in the same sentence(s) together

# load dictionary with pickle
import pickle 

# also AE is an option: 
AE = True

# GIJS = False

if not AE: 
    if GIJS: 
        # Numpymodel 
        with open('../weights/weightsDictNP1QB_gijs_500.pkl', 'rb') as fp: 
            parameterDictNP = pickle.load(fp)

        # Numpymodel 
        with open('../weights/weightsDictTK1QB_gijs_500.pkl', 'rb') as fp: 
            parameterDictTK = pickle.load(fp)

        # Pennylane model 
        with open('../weights/weightsDictPL1QB_gijs_500.pkl', 'rb') as fp: 
            parameterDictPL = pickle.load(fp)

    if not GIJS: 
        # Numpymodel 
        with open('../weights/weightsDictNP1QB_500.pkl', 'rb') as fp: 
            parameterDictNP = pickle.load(fp)

        # TKet model 
        with open('../weights/weightsDictTK1QB_500.pkl', 'rb') as fp: 
            parameterDictTK = pickle.load(fp)

        # Pennylane model 
        with open('../weights/weightsDictPL1QB_500.pkl', 'rb') as fp: 
            parameterDictPL = pickle.load(fp)

 

if AE: 
    # Numpymodel 
    with open('../weights/weightsDict_NP_1QB_AE_500.pkl', 'rb') as fp: 
        parameterDictNP = pickle.load(fp)

    # TKet model 
    with open('../weights/weightsDict_TK_1QB_AE_500_gijs.pkl', 'rb') as fp: 
        parameterDictTK = pickle.load(fp)

    # Pennylane model 
    with open('../weights/weightsDict_PL_1QB_AE_500_gijs.pkl', 'rb') as fp: 
        parameterDictPL = pickle.load(fp)


print(parameterDictNP)


{'application__n_0': 0.7566747098665764, 'application__n_1': 0.6747791281848307, 'application__n_2': 0.9214202298180648, 'application†_n__0': 0.5919034206812943, 'application†_n__1': 0.6637414290884049, 'application†_n__2': 0.7892905347081347, 'bakes__n.r@s@n.l_0': 0.48800293786770016, 'bakes__n.r@s@n.l_1': 1.2214454625039055, 'chef__n_0': 0.9767400239410216, 'chef__n_1': 0.3550526720199546, 'chef__n_2': 0.47422007328053306, 'chef†_n__0': 0.8815980065268059, 'chef†_n__1': 0.2784892377144183, 'chef†_n__2': 0.7748552658126453, 'cooks__n.r@s@n.l_0': 0.008643054206695661, 'cooks__n.r@s@n.l_1': 0.09576369738347457, 'dinner__n_0': -0.036798524618151134, 'dinner__n_1': 0.9444265545335363, 'dinner__n_2': 0.6149735278475615, 'dinner†_n__0': 0.7226261506940829, 'dinner†_n__1': 1.1698460558495352, 'dinner†_n__2': 0.3900996033194077, 'meal__n_0': 0.37272605689359567, 'meal__n_1': 0.9279550938835237, 'meal__n_2': 0.20875976681424938, 'meal†_n__0': 0.25024759254602763, 'meal†_n__1': 0.79902564654082

In [66]:
'''
apply the functions
'''

# parameterDict = parameterDictNP
parameterDict = parameterDictTK
# parameterDict = parameterDictPL

print(parameterDict)

naiveDMListForgetSubj = []
naiveDMListForgetObj  = []
gijsDMListForgetObj = []
gijsDMListForgetSubj = []
myDMListForgetSubj = []
myDMListForgetObj  = [] 

myFDListForgetSubj1QB = []
myFDListForgetObj1QB  = [] 

naiveFDListForgetSubj1QB = []
naiveFDListForgetObj1QB  = [] 
gijsFDListForgetSubj1QB = []
gijsFDListForgetObj1QB  = [] 


for nC in naiveCircuits: 

    subjResult = Main(listOfCircuits= [nC], parameterDict= parameterDict, wordsToForget=wordsToForgetSubj, amplitudeEncoded = AE)
    objResult = Main(listOfCircuits= [nC], parameterDict= parameterDict, wordsToForget=wordsToForgetObj, amplitudeEncoded = AE)

    naiveDMListForgetSubj.append(subjResult[0])
    naiveDMListForgetObj.append(objResult[0])

    naiveFDListForgetObj1QB.append(objResult[2])
    naiveFDListForgetSubj1QB.append(subjResult[2])


# commented this out

if GIJS:
    print('GIJS')
    for gC in gijsCircuits: 

        subjResult = Main(listOfCircuits= [gC], parameterDict= parameterDict, wordsToForget= ['person'], amplitudeEncoded = AE)
        objResult = Main(listOfCircuits= [gC], parameterDict= parameterDict, wordsToForget=wordsToForgetObj, amplitudeEncoded = AE)


        gijsDMListForgetObj.append(objResult[0])
        gijsDMListForgetSubj.append(subjResult[0])

        gijsFDListForgetObj1QB.append(objResult[2])
        gijsFDListForgetSubj1QB.append(subjResult[2])

print('MY')

for mC in myCircuits:

    subjResult = Main(listOfCircuits= [mC], parameterDict= parameterDict, wordsToForget=wordsToForgetSubj, amplitudeEncoded = AE)

    myDMListForgetSubj.append(subjResult[0])
    myFDListForgetSubj1QB.append(subjResult[2])


    objResult = Main(listOfCircuits= [mC], parameterDict= parameterDict, wordsToForget=wordsToForgetObj, amplitudeEncoded = AE)

    myDMListForgetObj.append(objResult[0])
    myFDListForgetObj1QB.append(objResult[2])
    # myDMList.append(Main(listOfCircuits= [mC], parameterDict= parameterDict, wordsToForget=wordsToForgetSubj)[0])


{'application__n_0': 1.5748560069820547, 'application__n_1': 0.6346459535519534, 'application__n_2': -1.00079375053177, 'application†_n__0': 0.31650490891726885, 'application†_n__1': 0.49180453302847077, 'application†_n__2': 1.4268844406270615, 'bakes__n.r@s@n.l_0': 2.0885700849408826, 'bakes__n.r@s@n.l_1': 0.1596371342036533, 'chef__n_0': 1.2898107533996523, 'chef__n_1': -1.1538447705418335, 'chef__n_2': 0.6457142159589875, 'chef†_n__0': 2.1517061233071315, 'chef†_n__1': 2.2757935852939903, 'chef†_n__2': -1.6991932086819672, 'cooks__n.r@s@n.l_0': -0.0027020889354133926, 'cooks__n.r@s@n.l_1': -4.000375879142329, 'dinner__n_0': 1.0283807213017215, 'dinner__n_1': -0.6952283266791935, 'dinner__n_2': -0.8261218955459253, 'dinner†_n__0': 1.0535415423823526, 'dinner†_n__1': 0.8485428679421526, 'dinner†_n__2': 2.01070743557533, 'meal__n_0': -1.8446019541081107, 'meal__n_1': 1.8711006519758975, 'meal__n_2': 0.9939039104268897, 'meal†_n__0': 1.3632503629225776, 'meal†_n__1': -1.2291139089186398

/home/bob/anaconda3/envs/code/lib/python3.9/site-packages/qiskit/visualization/circuit/matplotlib.py:266: FutureWarning: The default matplotlib drawer scheme will be changed to "iqp" in a following release. To silence this warning, specify the current default explicitly as style="clifford", or the new default as style="iqp".
  self._style, def_font_ratio = load_style(self._style)


removed measurement. 
trace out instead of measure, circuit is: 
     ┌───┐                    ┌───┐                                       »
q_0: ┤ H ├───────■────────────┤ H ├───────────────────────────────────────»
     ├───┤┌──────┴──────┐     └───┘     ┌───┐                             »
q_1: ┤ H ├┤ Rz(-17.035) ├───────■───────┤ H ├─────────────────────────────»
     ├───┤└─────────────┘┌──────┴──────┐├───┤┌─────────────┐┌────────────┐»
q_2: ┤ H ├───────────────┤ Rz(0.97971) ├┤ H ├┤ Rx(-8.5656) ├┤ Rz(7.7228) ├»
     └───┘               └─────────────┘└───┘└─────────────┘└────────────┘»
c: 2/═════════════════════════════════════════════════════════════════════»
                                                                          »
«                   
«q_0: ──────────────
«                   
«q_1: ──────────────
«     ┌────────────┐
«q_2: ┤ Rx(-4.115) ├
«     └────────────┘
«c: 2/══════════════
«                   
And traceList is:  [2]
debug, shape density matrix:  DensityMa

In [67]:
from qiskit.quantum_info import entropy, purity
print(myDMListForgetObj)
naiveEntropyListForgetSubj = []
naiveEntropyListForgetObj  = []
gijsEntropyListForgetObj  = []
gijsEntropyListForgetSubj = []
myEntropyListForgetSubj    = []
myEntropyListForgetObj     = []

for dM1 in naiveDMListForgetSubj: 
    # print(dM1)
    naiveEntropyListForgetSubj.append(entropy(dM1[0]))

for dM2 in naiveDMListForgetObj: 
    naiveEntropyListForgetObj.append(entropy(dM2[0]))

for dM in gijsDMListForgetObj: 
    gijsEntropyListForgetObj.append(entropy(dM[0], base = np.e))

for dM in gijsDMListForgetSubj: 
    gijsEntropyListForgetSubj.append(entropy(dM[0]))

for dM3 in myDMListForgetSubj: 
    myEntropyListForgetSubj.append(entropy(dM3[0]))

for dM4 in myDMListForgetObj: 
    myEntropyListForgetObj.append(entropy(dM4[0]))


# print(gijsDMListForgetObj[0])
# print(gijsEntropyListForgetObj)


[[DensityMatrix([[9.99977421e-01+1.03625281e-17j,
                4.42001889e-03-1.73654422e-03j],
               [4.42001889e-03+1.73654422e-03j,
                2.25794810e-05+6.35675516e-18j]],
              dims=(2,))], [DensityMatrix([[ 0.99177181-1.32485699e-17j, -0.03805619+3.18290685e-02j],
               [-0.03805619-3.18290685e-02j,  0.00822819-2.40200700e-19j]],
              dims=(2,))], [DensityMatrix([[9.99992666e-01-1.41500675e-18j,
                2.62065213e-03-6.62632976e-04j],
               [2.62065213e-03+6.62632976e-04j,
                7.33377353e-06-9.68651022e-19j]],
              dims=(2,))], [DensityMatrix([[ 0.99177181-1.32485699e-17j, -0.03805619+3.18290685e-02j],
               [-0.03805619-3.18290685e-02j,  0.00822819-2.40200700e-19j]],
              dims=(2,))], [DensityMatrix([[ 9.99977626e-01-1.74449248e-16j,
                -5.61860385e-04+4.57706838e-03j],
               [-5.61860385e-04-4.57706838e-03j,
                 2.23737082e-05-2.37436536e-18

In [68]:

# collect both entropy and fidelities 


# we now calculalte the fidelity of the dms with the respective pure density matrices corresponding to the both categories 

from qiskit.quantum_info import state_fidelity
from qiskit.quantum_info import partial_trace

iTMatrix = np.array([[1,0], [0,0]])
foodMatrix = np.array([[0,0], [0,1]])
iTDM = DensityMatrix(iTMatrix)
foodDM = DensityMatrix(foodMatrix)


naiveFidelitiesSubj = []
naiveFidelitiesObj = []

myFidelitiesSubj = []
myFidelitiesObj = []

gijsFidelitiesSubj = []
gijsFidelitiesObj = [] 

# 
# NAIVE!! 
# 

objList = []
subjList = []
for i in range(len(senseFulSentences)):

    
    if 'prepares' in senseFulSentences[i]:
        objList.append(naiveEntropyListForgetObj[i])
        subjList.append(naiveEntropyListForgetSubj[i])

        

        # # if the sentence is in the category food
        if 'meal' in senseFulSentences[i] or 'sauce' in senseFulSentences[i] or 'dinner' in senseFulSentences[i] or 'chef' in senseFulSentences[i] or 'bakes' in senseFulSentences[i] or 'cooks' in senseFulSentences[i]:
            naiveFidelitiesObj.append(naiveFDListForgetObj1QB[i][1])
            naiveFidelitiesSubj.append(naiveFDListForgetSubj1QB[i][1])
        else: 
            naiveFidelitiesObj.append(naiveFDListForgetObj1QB[i][0])
            naiveFidelitiesSubj.append(naiveFDListForgetSubj1QB[i][0])



test1 = []
test2 = []
# my sentences fidelities 
for i in range(len(myDMListForgetObj)):  

    if 'cooks' in mySentences[i] or 'bakes' in mySentences[i]: 
        print('one')
        myFidelitiesObj.append(myFDListForgetObj1QB[i][1])
        myFidelitiesSubj.append(myFDListForgetSubj1QB[i][1])
        test1.append(myFDListForgetObj1QB[i][1])
        test2.append(myFDListForgetSubj1QB[i][1])

    else: 
        print('two')
        test1.append(myFDListForgetObj1QB[i][0])
        test2.append(myFDListForgetSubj1QB[i][0])
        myFidelitiesObj.append(myFDListForgetObj1QB[i][0])
        myFidelitiesSubj.append(myFDListForgetSubj1QB[i][0])

print('sums')
print(sum(test1) / len(test1))
print(sum(test2) / len(test2))


# # gijs sentences fidelity 
for i in range(len(gijsDMListForgetObj)):  

#     objDM = gijsDMListForgetObj[i][0]
#     objDM = partial_trace(objDM, [1])

#     subjDM = gijsDMListForgetSubj[i][0]
#     subjDM = partial_trace(subjDM, [0])

    if 'chef' in gijsSentences[i]: 
        gijsFidelitiesObj.append(gijsFDListForgetObj1QB[i][1])
        gijsFidelitiesSubj.append(gijsFDListForgetSubj1QB[i][1])
    else: 
        gijsFidelitiesObj.append(gijsFDListForgetObj1QB[i][0])
        gijsFidelitiesSubj.append(gijsFDListForgetSubj1QB[i][0])



# 
# APPEND THE FIDELITIES AS DOWNSTAIRS!! 
# 

print('avg fidelities naive: ')
print('obj')
print(sum( naiveFidelitiesObj) / len(naiveFidelitiesObj))
print('subj')
print(sum( naiveFidelitiesSubj) / len(naiveFidelitiesSubj))


print('avg fidelities my sentences: ')
print('obj')
print(sum( myFidelitiesObj) / len(myFidelitiesObj))
print('subj')
print(sum( myFidelitiesSubj) / len(myFidelitiesSubj))

if GIJS: 
    print('avg fidelities gijs: ')
    print('obj')
    print(sum( gijsFidelitiesObj) / len(gijsFidelitiesObj))
    print('subj')
    print(sum( gijsFidelitiesSubj) / len(gijsFidelitiesSubj))


    
print('### naive sentencnes ###')
print('forget object: ')
print(sum(objList)/len(objList))
print('forget subject: ')
print(sum(subjList)/ len(subjList))
# print(myEntropyListForgetObj)

print('### my sentences ###')
print('forget object: ', sum(myEntropyListForgetObj)/len(myEntropyListForgetObj))
print('forget subject: ', sum(myEntropyListForgetSubj)/len(myEntropyListForgetSubj))


if GIJS:
    print('### gijs DMs ###')
    print('forget object: ') 
    print(sum(gijsEntropyListForgetObj)/ len(gijsEntropyListForgetObj))
    print('forget subject: ')
    print(sum(gijsEntropyListForgetSubj)/ len(gijsEntropyListForgetSubj))


# print fidelities! 

print(gijsDMListForgetObj)
print(gijsDMListForgetSubj)
print(gijsFDListForgetObj1QB)
print(gijsFDListForgetSubj1QB)


one
one
one
one
one
one
two
two
two
two
two
two
sums
0.966654654002938
0.9923576862289364
avg fidelities naive: 
obj
0.4979027853851871
subj
0.48721450276963624
avg fidelities my sentences: 
obj
0.966654654002938
subj
0.9923576862289364
avg fidelities gijs: 
obj
0.42688821033818414
subj
0.5379089161209823
### naive sentencnes ###
forget object: 
0.27310031036349625
forget subject: 
0.6332097479772819
### my sentences ###
forget object:  0.06503224581609846
forget subject:  0.014929852373067092
### gijs DMs ###
forget object: 
0.21949148088044748
forget subject: 
0.5216248175104276
[[DensityMatrix([[ 0.5372154 -4.24137089e-17j, -0.08952562+4.32472524e-01j],
               [-0.08952562-4.32472524e-01j,  0.4627846 -4.24137089e-18j]],
              dims=(2,))], [DensityMatrix([[ 0.5372154 -4.24137089e-17j, -0.08952562+4.32472524e-01j],
               [-0.08952562-4.32472524e-01j,  0.4627846 -4.24137089e-18j]],
              dims=(2,))], [DensityMatrix([[0.90268066+2.31249785e-17j, 0.081352

In [69]:
for i in range(len(mySentences)):

    
    print(mySentences[i])

    print(myFDListForgetObj1QB[i])

for i in range(len(gijsSentences)):

    print(gijsSentences[i])
    print(gijsFDListForgetObj1QB[i])

person prepares meal and cooks it
[2.2579481006159726e-05, 0.9999774205189936]
person prepares meal and bakes it
[0.00822819369927912, 0.9917718063007215]
person prepares dinner and cooks it
[7.33377353343692e-06, 0.9999926662264667]
person prepares dinner and bakes it
[0.00822819369927912, 0.9917718063007215]
person prepares sauce and cooks it
[2.2373708236055747e-05, 0.9999776262917645]
person prepares sauce and bakes it
[0.030847818437455828, 0.9691521815625448]
person prepares program and runs it
[0.9996368617378341, 0.0003631382621675168]
person prepares program and reads it
[0.9793278806417205, 0.020672119358280158]
person prepares software and runs it
[0.9996368617378341, 0.0003631382621675168]
person prepares software and reads it
[0.8475136016571168, 0.15248639834288352]
person prepares application and runs it
[0.9864068106740983, 0.013593189325901969]
person prepares application and reads it
[0.8346903243854388, 0.16530967561456053]
person prepares meal and chef does too
[0.4

In [70]:
'''
Results of Von Neumann Entropy 
'''

print('Naive entropy: ', naiveEntropyList)

print('Gijs entropy: ', gijsEntropyList)

print('My entropy:', myEntropyList)

print('Naive purity: ', naivePurityList)

print('Gijs purity: ', gijsPurityList)

print('My purity:', myPurityList)

NameError: name 'naiveEntropyList' is not defined

In [ ]:
'''
Calculate fidelity between two density matrices
'''

def Fidelity(rho, sigma): 

    # from sklearn.preprocessing import normalize

    # rho = normalize(rho, norm = 'l1')
    # sigma = normalize(sigma, norm = 'l1')

    print('trace rho: ', np.trace(rho))
    print('trace sigma: ', np.trace(sigma))

    # first approach 
    partI = np.matmul(sigma, np.sqrt(rho))
    partII = np.sqrt( np.matmul(np.sqrt(rho), partI) )
    partIII = np.trace(partII)
    


    # # second approach via trace norm
    # partIII = np.trace(abs(np.dot( np.sqrt(rho), np.sqrt(sigma)))) ** 2

    
    return abs(partIII)



In [ ]:
from qiskit.quantum_info import state_fidelity

print('fidelity between naive and gijs: ', Fidelity(naiveDMList[0][0].data, gijsDMList[0][0].data))

print('fidelity between naive and my: ', Fidelity(naiveDMList[0][0].data, myDMList[0][0].data))

print('fidelity between gijs and my: ', Fidelity(gijsDMList[0][0].data, myDMList[0][0].data))

print('Qiskit says the followin: ')
print('Naive, Gijs: ', state_fidelity(naiveDMList[0][0], gijsDMList[0][0]))
print('naive, me', state_fidelity(naiveDMList[0][0], myDMList[0][0]))


print('Trace Distance')
print('naive,Gijs: ', TraceDistance(naiveDMList[0][0], gijsDMList[0][0]))
print('naive, me', TraceDistance(naiveDMList[0][0], myDMList[0][0]))

print('naive, naive', TraceDistance(naiveDMList[0][0], naiveDMList[0][0]))

In [ ]:
def TraceDistance(rho, sigma):
    '''
    this is another measure to compare density matrices 
    ''' 
    newMatrix = rho - sigma

    absValue = np.sqrt(newMatrix)

    return 1/2 * np.trace(np.abs(newMatrix))


In [ ]:
'''
We continue by comparing the density matrices to how far away they are from the two Categories 
'''

iTMatrix = np.array([[1,0], [0,0]])
foodMatrix = np.array([[0,0], [0,1]])

In [ ]:
iTDM = DensityMatrix(iTMatrix)
foodDM = DensityMatrix(foodMatrix)

In [ ]:
print(iTDM)
print(foodDM)

In [ ]:
print(state_fidelity(myDMList[0][0], iTDM))

In [ ]:
def SpectralDecomposition(rho):

    eigenvalues, eigenvectors = np.linalg.eig(rho)
    # rhoDiagonalized = np.diag(eigenvalues)

    normalizedEvs = []
    for eV in eigenvectors: 
        normalizedEvs.append(eV / np.linalg.norm(eV))


    projectors = []

    for eV in normalizedEvs: 
        projectors.append(np.outer((eV), eV))

    return eigenvalues, projectors

    


In [ ]:
def SpiderAnd(rho, sigma): 
    '''
    ...
    '''
    return np.matmul(rho, sigma)

def Fuzz(rho, sigma): 
    '''
    ...
    '''
    eVList, projectors = SpectralDecomposition(sigma)

    print('testetsetse')
    print(projectors[0] * projectors[0])

    termOne = eVList[0] * ( np.matmul(np.matmul(projectors[0], rho), projectors[0])) 
    termTwo = eVList[1] * ( np.matmul(np.matmul(projectors[1], rho), projectors[1]))  

    return termOne + termTwo


def Phaser(rho, sigma): 
    '''
    ...
    '''
    return np.matmul(np.sqrt(sigma), np.matmul(rho, np.sqrt(sigma)))

In [ ]:
result = SpiderAnd()

In [ ]:
sentencePairOne  = ['person prepares meal', 'person cooks meal']

parameterDict = parameterDictTK

circuits = []
for i in range(len(sentencePairOne)): 
    circuits.append(NaiveDiag(sentencePairOne[i]))

dMList = []

for nC in circuits: 
    dMList.append(Main(listOfCircuits= [nC], parameterDict= parameterDict, wordsToForget=wordsToForgetObj)[0])

In [ ]:
dM1 = dMList[0][0]
dM2 = dMList[1][0]

print(entropy(dM1))
print(entropy(dM2))



In [ ]:
test1 = SpiderAnd(dM1, dM2)
test2 = Fuzz(dM1, dM2)
test3 = Phaser(dM1, dM2)

print(np.trace(test2).imag)

test2 = test2 / np.sqrt(np.trace(test2).real**2 + np.trace(test2).imag**2)
test1 = test1 / np.sqrt(np.trace(test1).real**2 + np.trace(test1).imag**2)
test3 = test3 / np.sqrt(np.trace(test3).real**2 + np.trace(test3).imag**2)

# test1 = test1.real

print(test1)
print(test2)
print(test3)

print(np.trace(test2))
print(np.trace(test1))
print(np.trace(test3))

In [ ]:

print(np.trace(dM2))

test2 = test2.real

dmNew1 = DensityMatrix(test1)

dmNew2 = DensityMatrix(test2)

dmNew3 = DensityMatrix(test3)

print(dmNew1)
print(np.trace(dmNew2))


print(purity(dM1), purity(dM2))

print(purity(dmNew1, validate= False))
print(purity(dmNew2, validate= False))
print(purity(dmNew3, validate= False))

# print(entropy(dmNew1))
# print(entropy(dmNew2))
# print(entropy(dmNew3))

In [ ]:
print(Fidelity(foodDM, dM1)) 
print(Fidelity(foodDM, dM2)) 

print(Fidelity(foodDM, dmNew1)) 
print(Fidelity(foodDM, dmNew2)) 
print(Fidelity(foodDM, dmNew3)) 